In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
import pydicom
import cv2

data_dir = r'D:\Alzheimers\Task1\Task1-Images\fMRI\Images'
batch_size = 8
image_size = (96, 96)
num_epochs = 5  
num_classes = 5  

def load_and_preprocess_images_and_labels(directory):
    image_data = []
    labels = []

    for filename in os.listdir(directory):
        if filename.endswith(".dcm"):
            dcm_path = os.path.join(directory, filename)
            try:
                dcm_data = pydicom.dcmread(dcm_path)
                img = dcm_data.pixel_array.astype(np.float32) / 255.0
                img = cv2.resize(img, image_size)
                image_data.append(img)
                
                filename_parts = filename.split('_')
                class_name = filename_parts[-1].split('.')[0]
                
                class_mapping = {'CN': 0, 'AD': 1, 'MCI': 2, 'EMCI': 3, 'LMCI': 4}
                label = class_mapping.get(class_name, -1) 
                
                if label != -1:
                    labels.append(label)
                else:
                    print(f"Unknown class name: {class_name}")
            except Exception as e:
                print(f"Error processing file {dcm_path}: {str(e)}")

    return np.array(image_data), np.array(labels)


# Load and preprocess all data from the directory
all_images, all_labels = load_and_preprocess_images_and_labels(data_dir)

# Split the data into training and testing sets (80% training, 20% testing)
train_images, test_images, train_labels, test_labels = train_test_split(
    all_images, all_labels, test_size=0.2, random_state=42
)

# Expand grayscale images to three channels (RGB)
train_images = np.repeat(train_images[:, :, :, np.newaxis], 3, axis=-1)
test_images = np.repeat(test_images[:, :, :, np.newaxis], 3, axis=-1)

# Convert labels to one-hot encoding
train_labels = tf.keras.utils.to_categorical(train_labels, num_classes)
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes)

# Calculate class weights to handle class imbalance
class_weights = class_weight.compute_sample_weight('balanced', train_labels.argmax(axis=1))
class_weights = dict(enumerate(class_weights))

# Create a data generator for training images with more augmentation
train_data_generator = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load the VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(96, 96, 3))
# or, alternatively, use InceptionV3
# base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(96, 96, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)  # Add dropout for regularization
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze some layers of the base model (you can experiment with the number of layers to freeze)
for layer in base_model.layers[:15]:
    layer.trainable = False

# Compile the model with a lower learning rate
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model using the data generator
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    history = model.fit(
        train_data_generator.flow(train_images, train_labels, batch_size=batch_size),
        steps_per_epoch=len(train_images) // batch_size,
        epochs=1,
        class_weight=class_weights,  # Apply class weights
        verbose=1
    )

# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(test_images, test_labels, verbose=2)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# Define the path where you want to save the model
save_path = r'D:\Alzheimers\Models\fMRI_Task1.h5'  # Replace with your desired path

# Save the model to the specified location
model.save(save_path)

Epoch 1/5
16019/16019 [==============================] - 3672s 229ms/step - loss: 0.4871 - accuracy: 0.7767
Epoch 2/5
16019/16019 [==============================] - 3670s 229ms/step - loss: 0.0874 - accuracy: 0.9684
Epoch 3/5
16019/16019 [==============================] - 3678s 230ms/step - loss: 0.0563 - accuracy: 0.9804
Epoch 4/5
16019/16019 [==============================] - 3658s 228ms/step - loss: 0.0447 - accuracy: 0.9852
Epoch 5/5
16019/16019 [==============================] - 3670s 229ms/step - loss: 0.0400 - accuracy: 0.9874
1002/1002 - 407s - loss: 0.0038 - accuracy: 0.9989 - 407s/epoch - 407ms/step
Test Accuracy: 99.89%


In [4]:
# Define the path where you want to save the model
model_save_path = r'D:\Alzheimers\Task1\Task1-Images\fMRI\Model'  # Change this path as needed

# Save the trained model
model.save(model_save_path)

print("Model saved successfully.")


INFO:tensorflow:Assets written to: D:\Alzheimers\Task1\Task1-Images\fMRI\Model\assets


INFO:tensorflow:Assets written to: D:\Alzheimers\Task1\Task1-Images\fMRI\Model\assets


Model saved successfully.


In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
import pydicom
import cv2

data_dir = r'D:\Alzheimers\Task1\Task1-Images\fMRI\Images'
batch_size = 8
image_size = (96, 96)
num_epochs = 5
num_classes = 5

def load_and_preprocess_images_and_labels(directory):
    image_data = []
    labels = []

    for filename in os.listdir(directory):
        if filename.endswith(".dcm"):
            dcm_path = os.path.join(directory, filename)
            try:
                dcm_data = pydicom.dcmread(dcm_path)
                img = dcm_data.pixel_array.astype(np.float32) / 255.0
                img = cv2.resize(img, image_size)
                image_data.append(img)

                filename_parts = filename.split('_')
                class_name = filename_parts[-1].split('.')[0]

                class_mapping = {'CN': 0, 'AD': 1, 'MCI': 2, 'EMCI': 3, 'LMCI': 4}
                label = class_mapping.get(class_name, -1)

                if label != -1:
                    labels.append(label)
                else:
                    print(f"Unknown class name: {class_name}")
            except Exception as e:
                print(f"Error processing file {dcm_path}: {str(e)}")

    return np.array(image_data), np.array(labels)

# Load and preprocess all data from the directory
all_images, all_labels = load_and_preprocess_images_and_labels(data_dir)

# Split the data into training and testing sets (80% training, 20% testing)
train_images, test_images, train_labels, test_labels = train_test_split(
    all_images, all_labels, test_size=0.2, random_state=42
)

# Expand grayscale images to three channels (RGB)
train_images = np.repeat(train_images[:, :, :, np.newaxis], 3, axis=-1)
test_images = np.repeat(test_images[:, :, :, np.newaxis], 3, axis=-1)

# Convert labels to one-hot encoding
train_labels = tf.keras.utils.to_categorical(train_labels, num_classes)
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes)

# Calculate class weights to handle class imbalance
class_weights = class_weight.compute_sample_weight('balanced', train_labels.argmax(axis=1))
class_weights = dict(enumerate(class_weights))

# Create a data generator for training images with more augmentation
train_data_generator = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load the InceptionV3 model
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(96, 96, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)  # Add dropout for regularization
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze some layers of the base model (you can experiment with the number of layers to freeze)
for layer in base_model.layers[:15]:
    layer.trainable = False

# Compile the model with a lower learning rate
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model using the data generator
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    history = model.fit(
        train_data_generator.flow(train_images, train_labels, batch_size=batch_size),
        steps_per_epoch=len(train_images) // batch_size,
        epochs=1,
        class_weight=class_weights,  # Apply class weights
        verbose=1
    )

# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(test_images, test_labels, verbose=2)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

87910968/87910968 [==============================] - 8s 0us/step
Epoch 1/5
16019/16019 [==============================] - 5067s 315ms/step - loss: 1.2420 - accuracy: 0.3803
Epoch 2/5
16019/16019 [==============================] - 5911s 369ms/step - loss: 0.8882 - accuracy: 0.5893
Epoch 3/5
16019/16019 [==============================] - 4798s 300ms/step - loss: 0.5311 - accuracy: 0.7794
Epoch 4/5
16019/16019 [==============================] - 4733s 295ms/step - loss: 0.3546 - accuracy: 0.8626
Epoch 5/5
16019/16019 [==============================] - 4717s 294ms/step - loss: 0.2522 - accuracy: 0.9060
1002/1002 - 108s - loss: 0.5545 - accuracy: 0.9644 - 108s/epoch - 107ms/step
Test Accuracy: 96.44%


In [5]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
import pydicom
import cv2

data_dir = r'D:\Alzheimers\Task1\Task1-Images\MRI\Images'
batch_size = 32
image_size = (224, 224)
num_epochs = 5
num_classes = 5

def load_and_preprocess_images_and_labels(directory):
    image_data = []
    labels = []

    for filename in os.listdir(directory):
        if filename.endswith(".dcm"):
            dcm_path = os.path.join(directory, filename)
            try:
                dcm_data = pydicom.dcmread(dcm_path)
                img = dcm_data.pixel_array.astype(np.float32) / 255.0
                img = cv2.resize(img, image_size)
                image_data.append(img)

                filename_parts = filename.split('_')
                class_name = filename_parts[-1].split('.')[0]

                class_mapping = {'CN': 0, 'AD': 1, 'MCI': 2, 'EMCI': 3, 'LMCI': 4}
                label = class_mapping.get(class_name, -1)

                if label != -1:
                    labels.append(label)
                else:
                    print(f"Unknown class name: {class_name}")
            except Exception as e:
                print(f"Error processing file {dcm_path}: {str(e)}")

    return np.array(image_data), np.array(labels)

# Load and preprocess all data from the directory
all_images, all_labels = load_and_preprocess_images_and_labels(data_dir)

# Split the data into training and testing sets (80% training, 20% testing)
train_images, test_images, train_labels, test_labels = train_test_split(
    all_images, all_labels, test_size=0.2, random_state=42
)

# Expand grayscale images to three channels (RGB)
train_images = np.repeat(train_images[:, :, :, np.newaxis], 3, axis=-1)
test_images = np.repeat(test_images[:, :, :, np.newaxis], 3, axis=-1)

# Convert labels to one-hot encoding
train_labels = tf.keras.utils.to_categorical(train_labels, num_classes)
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes)

# Calculate class weights to handle class imbalance
class_weights = class_weight.compute_sample_weight('balanced', train_labels.argmax(axis=1))
class_weights = dict(enumerate(class_weights))

# Create a data generator for training images with more augmentation
train_data_generator = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load the InceptionV3 model
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)  # Add dropout for regularization
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze some layers of the base model (you can experiment with the number of layers to freeze)
for layer in base_model.layers[:15]:
    layer.trainable = False

# Compile the model with a lower learning rate
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model using the data generator
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    history = model.fit(
        train_data_generator.flow(train_images, train_labels, batch_size=batch_size),
        steps_per_epoch=len(train_images) // batch_size,
        epochs=1,
        class_weight=class_weights,  # Apply class weights
        verbose=1
    )

# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(test_images, test_labels, verbose=2)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Epoch 1/5
421/421 [==============================] - 1174s 3s/step - loss: 1.3600 - accuracy: 0.3152
Epoch 2/5
421/421 [==============================] - 1159s 3s/step - loss: 1.3128 - accuracy: 0.3422
Epoch 3/5
102/421 [======>.......................] - ETA: 14:33 - loss: 1.2985 - accuracy: 0.3498

KeyboardInterrupt: 

In [3]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
import pydicom
import cv2

data_dir = r'D:\Alzheimers\Task1\Task1-Images\PET\Images'
batch_size = 32
image_size = (128, 128)
num_epochs = 20
num_classes = 5  

def load_and_preprocess_images_and_labels(directory):
    image_data = []
    labels = []

    for filename in os.listdir(directory):
        if filename.endswith(".dcm"):
            dcm_path = os.path.join(directory, filename)
            try:
                dcm_data = pydicom.dcmread(dcm_path)
                img = dcm_data.pixel_array.astype(np.float32) / 255.0
                img = cv2.resize(img, image_size)
                image_data.append(img)
                
                filename_parts = filename.split('_')
                class_name = filename_parts[-1].split('.')[0]
                
                class_mapping = {'CN': 0, 'AD': 1, 'MCI': 2, 'EMCI': 3, 'LMCI': 4}
                label = class_mapping.get(class_name, -1) 
                
                if label != -1:
                    labels.append(label)
                else:
                    print(f"Unknown class name: {class_name}")
            except Exception as e:
                print(f"Error processing file {dcm_path}: {str(e)}")

    return np.array(image_data), np.array(labels)


# Load and preprocess all data from the directory
all_images, all_labels = load_and_preprocess_images_and_labels(data_dir)

# Split the data into training and testing sets (80% training, 20% testing)
train_images, test_images, train_labels, test_labels = train_test_split(
    all_images, all_labels, test_size=0.2, random_state=42
)

# Expand grayscale images to three channels (RGB)
train_images = np.repeat(train_images[:, :, :, np.newaxis], 3, axis=-1)
test_images = np.repeat(test_images[:, :, :, np.newaxis], 3, axis=-1)

# Convert labels to one-hot encoding
train_labels = tf.keras.utils.to_categorical(train_labels, num_classes)
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes)

# Calculate class weights to handle class imbalance
class_weights = class_weight.compute_sample_weight('balanced', train_labels.argmax(axis=1))
class_weights = dict(enumerate(class_weights))

# Create a data generator for training images with more augmentation
train_data_generator = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load the VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
# or, alternatively, use InceptionV3
# base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(96, 96, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)  # Add dropout for regularization
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze some layers of the base model (you can experiment with the number of layers to freeze)
for layer in base_model.layers[:15]:
    layer.trainable = False

# Compile the model with a lower learning rate
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model using the data generator
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    history = model.fit(
        train_data_generator.flow(train_images, train_labels, batch_size=batch_size),
        steps_per_epoch=len(train_images) // batch_size,
        epochs=1,
        class_weight=class_weights,  # Apply class weights
        verbose=1
    )

# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(test_images, test_labels, verbose=2)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


Epoch 1/20
1234/1234 [==============================] - 1316s 1s/step - loss: 0.8454 - accuracy: 0.4485
Epoch 2/20
1234/1234 [==============================] - 1327s 1s/step - loss: 0.7213 - accuracy: 0.5187
Epoch 3/20
1234/1234 [==============================] - 1317s 1s/step - loss: 0.6448 - accuracy: 0.5687
Epoch 4/20
1234/1234 [==============================] - 1313s 1s/step - loss: 0.5765 - accuracy: 0.6185
Epoch 5/20
1234/1234 [==============================] - 1316s 1s/step - loss: 0.5117 - accuracy: 0.6628
Epoch 6/20
1234/1234 [==============================] - 1315s 1s/step - loss: 0.4610 - accuracy: 0.6995
Epoch 7/20
1234/1234 [==============================] - 1317s 1s/step - loss: 0.4099 - accuracy: 0.7356
Epoch 8/20
1234/1234 [==============================] - 1320s 1s/step - loss: 0.3682 - accuracy: 0.7643
Epoch 9/20
1234/1234 [==============================] - 1319s 1s/step - loss: 0.3338 - accuracy: 0.7894
Epoch 10/20
1234/1234 [==============================] - 1320s 1

In [9]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB3  # Import EfficientNetB3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
import pydicom
import cv2

data_dir = r'D:\Alzheimers\Task1\Task1-Images\PET\Images'
batch_size = 32
image_size = (128, 128)
num_epochs = 20
num_classes = 5  

def load_and_preprocess_images_and_labels(directory):
    image_data = []
    labels = []

    for filename in os.listdir(directory):
        if filename.endswith(".dcm"):
            dcm_path = os.path.join(directory, filename)
            try:
                dcm_data = pydicom.dcmread(dcm_path)
                img = dcm_data.pixel_array.astype(np.float32) / 255.0
                img = cv2.resize(img, image_size)
                image_data.append(img)
                
                filename_parts = filename.split('_')
                class_name = filename_parts[-1].split('.')[0]
                
                class_mapping = {'CN': 0, 'AD': 1, 'MCI': 2, 'EMCI': 3, 'LMCI': 4}
                label = class_mapping.get(class_name, -1) 
                
                if label != -1:
                    labels.append(label)
                else:
                    print(f"Unknown class name: {class_name}")
            except Exception as e:
                print(f"Error processing file {dcm_path}: {str(e)}")

    return np.array(image_data), np.array(labels)

# Load and preprocess all data from the directory
all_images, all_labels = load_and_preprocess_images_and_labels(data_dir)

# Split the data into training and testing sets (80% training, 20% testing)
train_images, test_images, train_labels, test_labels = train_test_split(
    all_images, all_labels, test_size=0.2, random_state=42
)

# Expand grayscale images to three channels (RGB)
train_images = np.repeat(train_images[:, :, :, np.newaxis], 3, axis=-1)
test_images = np.repeat(test_images[:, :, :, np.newaxis], 3, axis=-1)

# Convert labels to one-hot encoding
train_labels = tf.keras.utils.to_categorical(train_labels, num_classes)
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes)

# Calculate class weights to handle class imbalance
class_weights = class_weight.compute_sample_weight('balanced', train_labels.argmax(axis=1))
class_weights = dict(enumerate(class_weights))

# Create a data generator for training images with more augmentation
train_data_generator = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load the EfficientNetB3 model
base_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape=(128, 128, 3))  # Use EfficientNetB3

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)  # Add dropout for regularization
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze some layers of the base model (you can experiment with the number of layers to freeze)
for layer in base_model.layers[:15]:
    layer.trainable = False

# Compile the model with a lower learning rate
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model using the data generator
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    history = model.fit(
        train_data_generator.flow(train_images, train_labels, batch_size=batch_size),
        steps_per_epoch=len(train_images) // batch_size,
        epochs=1,
        class_weight=class_weights,  # Apply class weights
        verbose=1
    )

# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(test_images, test_labels, verbose=2)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


Epoch 1/20
1234/1234 [==============================] - 2637s 2s/step - loss: 0.8447 - accuracy: 0.4454
Epoch 2/20
1234/1234 [==============================] - 2617s 2s/step - loss: 0.6978 - accuracy: 0.5397
Epoch 3/20
1234/1234 [==============================] - 2596s 2s/step - loss: 0.5840 - accuracy: 0.6156
Epoch 4/20
1234/1234 [==============================] - 2601s 2s/step - loss: 0.4834 - accuracy: 0.6883
Epoch 5/20
1234/1234 [==============================] - 2607s 2s/step - loss: 0.4065 - accuracy: 0.7415
Epoch 6/20
1234/1234 [==============================] - 2609s 2s/step - loss: 0.3403 - accuracy: 0.7875
Epoch 7/20
1234/1234 [==============================] - 2612s 2s/step - loss: 0.2936 - accuracy: 0.8203
Epoch 8/20
1234/1234 [==============================] - 2597s 2s/step - loss: 0.2589 - accuracy: 0.8438
Epoch 9/20
1234/1234 [==============================] - 2613s 2s/step - loss: 0.2311 - accuracy: 0.8602
Epoch 10/20
1234/1234 [==============================] - 2610s 2

In [11]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.applications import VGG16, InceptionV3, EfficientNetB3
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
import pydicom
import cv2
from keras_tuner.tuners import RandomSearch
import kerastuner


# Set your data directory, batch size, image size, number of epochs, and number of classes
data_dir = r'D:\Alzheimers\Task1\Task1-Images\MRI\Images'
batch_size = 32
image_size = (128, 128)
num_epochs = 20
num_classes = 5

# Function to load and preprocess images and labels
def load_and_preprocess_images_and_labels(directory):
    image_data = []
    labels = []

    for filename in os.listdir(directory):
        if filename.endswith(".dcm"):
            dcm_path = os.path.join(directory, filename)
            try:
                dcm_data = pydicom.dcmread(dcm_path)
                img = dcm_data.pixel_array.astype(np.float32) / 255.0
                img = cv2.resize(img, image_size)
                image_data.append(img)
                
                filename_parts = filename.split('_')
                class_name = filename_parts[-1].split('.')[0]
                
                class_mapping = {'CN': 0, 'AD': 1, 'MCI': 2, 'EMCI': 3, 'LMCI': 4}
                label = class_mapping.get(class_name, -1) 
                
                if label != -1:
                    labels.append(label)
                else:
                    print(f"Unknown class name: {class_name}")
            except Exception as e:
                print(f"Error processing file {dcm_path}: {str(e)}")

    return np.array(image_data), np.array(labels)

# Load and preprocess all data from the directory
all_images, all_labels = load_and_preprocess_images_and_labels(data_dir)

# Split the data into training and testing sets
train_images, test_images, train_labels, test_labels = train_test_split(
    all_images, all_labels, test_size=0.2, random_state=42
)

# Expand grayscale images to three channels (RGB)
train_images = np.repeat(train_images[:, :, :, np.newaxis], 3, axis=-1)
test_images = np.repeat(test_images[:, :, :, np.newaxis], 3, axis=-1)

# Convert labels to one-hot encoding
train_labels = tf.keras.utils.to_categorical(train_labels, num_classes)
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes)

# Calculate class weights to handle class imbalance
class_weights = class_weight.compute_sample_weight('balanced', train_labels.argmax(axis=1))
class_weights = dict(enumerate(class_weights))

# Create a data generator for training images with more augmentation
train_data_generator = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Define a function to create and compile a model
def build_model(hp):
    # Choose a transfer learning model architecture
    transfer_model = hp.Choice('transfer_model', ['InceptionV3', 'EfficientNetB3'])
    if transfer_model == 'InceptionV3':
        base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
    elif transfer_model == 'EfficientNetB3':
        base_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    
    # Add regularization (L2 regularization)
    x = Dense(1024, activation='relu', kernel_regularizer=l2(hp.Float('regularization', 1e-5, 1e-3, sampling='log')))(x)
    
    # Add dropout for regularization
    x = Dropout(hp.Float('dropout', 0.2, 0.5, step=0.1))(x)
    predictions = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=predictions)

    # Freeze some layers of the base model
    for layer in base_model.layers[:15]:
        layer.trainable = False

    # Compile the model with a variable learning rate
    model.compile(
        optimizer=Adam(learning_rate=hp.Float('learning_rate', 1e-5, 1e-2, sampling='log')),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Instantiate the RandomSearch tuner
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,  # Number of hyperparameter combinations to try
    directory='tuner_results',  # Directory to store results
    project_name='alzheimer'  # Name of the project
)

# Search for the best hyperparameters
tuner.search(train_images, train_labels, epochs=num_epochs, validation_split=0.2, class_weight=class_weights, callbacks=[early_stopping])

# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]

# Train the best model with early stopping
best_model.fit(
    train_data_generator.flow(train_images, train_labels, batch_size=batch_size),
    steps_per_epoch=len(train_images) // batch_size,
    epochs=num_epochs,
    validation_data=(test_images, test_labels),
    class_weight=class_weights,  # Apply class weights
    callbacks=[early_stopping]
)

# Evaluate the best model on the test dataset
test_loss, test_accuracy = best_model.evaluate(test_images, test_labels, verbose=2)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


Trial 7 Complete [01h 45m 13s]
val_accuracy: 0.6600000262260437

Best val_accuracy So Far: 0.7237036824226379
Total elapsed time: 16h 21m 40s

Search: Running Trial #8

Value             |Best Value So Far |Hyperparameter
EfficientNetB3    |EfficientNetB3    |transfer_model
8.372e-05         |0.00019545        |regularization
0.4               |0.3               |dropout
2.9132e-05        |8.2548e-05        |learning_rate

Epoch 1/20
338/338 [==============================] - 792s 2s/step - loss: 1.5001 - accuracy: 0.3043 - val_loss: 1.6089 - val_accuracy: 0.3204
Epoch 2/20
338/338 [==============================] - 756s 2s/step - loss: 1.4220 - accuracy: 0.3383 - val_loss: 1.5390 - val_accuracy: 0.3441
Epoch 3/20
338/338 [==============================] - 755s 2s/step - loss: 1.3660 - accuracy: 0.3662 - val_loss: 1.4821 - val_accuracy: 0.3781
Epoch 4/20
338/338 [==============================] - 757s 2s/step - loss: 1.3089 - accuracy: 0.4054 - val_loss: 1.4511 - val_accuracy: 0.4104
E

KeyboardInterrupt: 

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load your three saved models (MRI, fMRI, and PET)
# You'll need to replace these with the actual paths to your saved models.
path_to_mri_model = 'path_to_saved_mri_model.pkl'
path_to_fmri_model = 'path_to_saved_fmri_model.pkl'
path_to_pet_model = 'path_to_saved_pet_model.pkl'

# Load your data (features and labels)
# Replace X and y with your actual data.
X, y = load_data()  # You need to implement load_data() based on your dataset.

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Load the saved models
mri_model = load_model(path_to_mri_model)  # You need to implement load_model() based on your model saving method.
fmri_model = load_model(path_to_fmri_model)
pet_model = load_model(path_to_pet_model)

# Create a list of tuples where each tuple contains a name for the model and the model itself
# You can use any name you prefer for each model.
estimators = [
    ('mri', mri_model),
    ('fmri', fmri_model),
    ('pet', pet_model)
]

# Create a VotingClassifier with majority voting
voting_classifier = VotingClassifier(estimators=estimators, voting='hard')

# Fit the VotingClassifier on the training data
voting_classifier.fit(X_train, y_train)

# Make predictions on the test data
y_pred = voting_classifier.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Voting Classifier Accuracy: {accuracy * 100:.2f}%')


In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.applications import InceptionV3, EfficientNetB3
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
import pydicom
import cv2
from kerastuner.tuners import RandomSearch

# Set your data directory, batch size, image size, number of epochs, and number of classes
data_dir = r'D:\Alzheimers\Task1\Task1-Images\MRI\Images'
batch_size = 32
image_size = (128, 128)
num_epochs = 20
num_classes = 5

# Function to load and preprocess images and labels
def load_and_preprocess_images_and_labels(directory):
    image_data = []
    labels = []

    for filename in os.listdir(directory):
        if filename.endswith(".dcm"):
            dcm_path = os.path.join(directory, filename)
            try:
                dcm_data = pydicom.dcmread(dcm_path)
                img = dcm_data.pixel_array.astype(np.float32) / 255.0
                img = cv2.resize(img, image_size)
                image_data.append(img)
                
                filename_parts = filename.split('_')
                class_name = filename_parts[-1].split('.')[0]
                
                class_mapping = {'CN': 0, 'AD': 1, 'MCI': 2, 'EMCI': 3, 'LMCI': 4}
                label = class_mapping.get(class_name, -1) 
                
                if label != -1:
                    labels.append(label)
                else:
                    print(f"Unknown class name: {class_name}")
            except Exception as e:
                print(f"Error processing file {dcm_path}: {str(e)}")

    return np.array(image_data), np.array(labels)

# Load and preprocess all data from the directory
all_images, all_labels = load_and_preprocess_images_and_labels(data_dir)

# Split the data into training and testing sets
train_images, test_images, train_labels, test_labels = train_test_split(
    all_images, all_labels, test_size=0.2, random_state=42
)

# Expand grayscale images to three channels (RGB)
train_images = np.repeat(train_images[:, :, :, np.newaxis], 3, axis=-1)
test_images = np.repeat(test_images[:, :, :, np.newaxis], 3, axis=-1)

# Convert labels to one-hot encoding
train_labels = tf.keras.utils.to_categorical(train_labels, num_classes)
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes)

# Calculate class weights to handle class imbalance
class_weights = class_weight.compute_sample_weight('balanced', train_labels.argmax(axis=1))
class_weights = dict(enumerate(class_weights))

# Create a data generator for training images with more augmentation
train_data_generator = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Define a function to create and compile a model
def build_model(hp):
    # Choose a transfer learning model architecture
    transfer_model = hp.Choice('transfer_model', ['InceptionV3', 'EfficientNetB3'])
    if transfer_model == 'InceptionV3':
        base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
    elif transfer_model == 'EfficientNetB3':
        base_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    
    # Add regularization (L2 regularization)
    x = Dense(1024, activation='relu', kernel_regularizer=l2(hp.Float('regularization', 1e-5, 1e-3, sampling='log')))(x)
    
    # Add dropout for regularization
    x = Dropout(hp.Float('dropout', 0.2, 0.5, step=0.1))(x)
    predictions = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=predictions)

    # Freeze some layers of the base model
    for layer in base_model.layers[:15]:
        layer.trainable = False

    # Compile the model with a variable learning rate
    model.compile(
        optimizer=Adam(learning_rate=hp.Float('learning_rate', 1e-5, 1e-2, sampling='log')),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Define a ModelCheckpoint callback to save the best model
checkpoint_filepath = 'MRI_Task1.h5'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_best_only=True,
    monitor='val_accuracy',  # Monitor validation accuracy
    mode='max',  # Save the highest accuracy models
    verbose=1  # Display a message when the best model is saved
)

# Instantiate the RandomSearch tuner
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,  # Number of hyperparameter combinations to try
    directory='tuner_results',  # Directory to store results
    project_name='alzheimer'  # Name of the project
)

# Search for the best hyperparameters
tuner.search(train_images, train_labels, epochs=num_epochs, validation_split=0.2, class_weight=class_weights, callbacks=[early_stopping, model_checkpoint_callback])

# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]

# Evaluate the best model on the test dataset
test_loss, test_accuracy = best_model.evaluate(test_images, test_labels, verbose=2)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


Using TensorFlow backend


C:\Users\prana\AppData\Local\Temp\ipykernel_6068\912037774.py:15: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch



Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
InceptionV3       |InceptionV3       |transfer_model
2.4101e-05        |2.4101e-05        |regularization
0.3               |0.3               |dropout
1.7359e-05        |1.7359e-05        |learning_rate

Epoch 1/20
 73/338 [=====>........................] - ETA: 4:15 - loss: 1.5099 - accuracy: 0.2868

KeyboardInterrupt: 